In [ ]:
import pandas as pd

import re, json, glob

from dateutil.parser import parse as date_parse

In [ ]:
json_template = "speed_results/iperf_{}_{}mbit.json"
dl_dict = {"DL" : "Download", "UL" : "Upload"}

data = []
for f in glob.glob("speed_results/*L*json"):
    
    ud    = re.sub(r".*_([UD]L)_.*", r"\1", f)
    ud    = dl_dict[ud]
    speed = float(re.sub(r".*[UD]L_(.*)mbit.json", r"\1", f))
    
    with open(f) as fo: j = json.load(fo)

    mbits_per_s = j["end"]["sum_received"]["bits_per_second"] / 1e6
        
    data.append([ud, speed, mbits_per_s])


In [ ]:
cols = ["io", "tc_set", "iperf_res"]

df = pd.DataFrame(data, columns = cols)
df = df.pivot(index = "tc_set", values = "iperf_res", columns = "io")

df["DL Ratio"] = df["Download"] / df.index
df["UL Ratio"] = df["Upload"] / df.index

In [ ]:
fig, ax = plt.subplots(figsize = (3, 3))

df[["Download", "Upload"]].plot(ax = ax, color = [chred, chgrey])

ax.plot([0, 10], [0, 10], color = "k", lw = 1, ls = ":")

ax.set_xlim(0, 2)
ax.set_ylim(0, 2)

ax.set_xlabel("Input Bandwidth [Mbps]\n(tc)")
ax.set_ylabel("Measured Bandwidth [Mbps]\n(iperf3)")

leg = ax.legend(frameon = False)
fig.savefig("iperf_tc_xy.pdf")

In [ ]:
fig, ax = plt.subplots(figsize = (3, 3))

df[["DL Ratio", "UL Ratio"]]\
  .rename(columns = {"DL Ratio" : "Download", "UL Ratio" : "Upload"})\
  .plot(ax = ax, ms = 4, marker = '.', color = [chred, chgrey])

ax.plot([0, 2], [1, 1], c = "k", lw = 1, ls = ":")

ax.set_xlim(0, 2)
ax.set_ylim(0.90, 1.05)

ax.set_xlabel("Input Bandwidth [Mbps]\n(tc)")
ax.set_ylabel("Fraction of Input Bandwidth\n(iperf3)")

leg = ax.legend(frameon = False)
fig.savefig("iperf_tc_ratio.pdf")

### Zoom Metrics

In [ ]:
def get_meeting_qos():
    
    with open("meeting_metrics.json") as f:
        j = json.load(f)
        
    qos_values = []

    qos = j["user_qos"]

    for m in qos:

        if not m["video_output"]["bitrate"]: continue 

        ts      = int(date_parse(m["date_time"]).timestamp())

        bitrate = int(m["video_output"]["bitrate"].replace(" kbps", "")) / 1000
        loss    = float(m["video_output"]["avg_loss"].replace("%", ""))
        fps     = int(m["video_output"]["frame_rate"].replace(" fps", ""))
        res     = m["video_output"]["resolution"]
        resx, resy = [int(v) for v in res.split("*")]

        qos_values.append([ts, bitrate, loss, resx, resy, fps])


    columns = ["ts", "bitrate", "loss", "resx", "resy", "fps"]
    df = pd.DataFrame(qos_values, columns = columns)

    # df.ts = pd.to_datetime(df.ts, unit = "s")
    # df.set_index("ts", inplace = True)
    
    df.sort_values(by = "ts")
    
    return df

In [ ]:
bw  = pd.read_csv("speed_cycle.csv").sort_values("ts")
qos = get_meeting_qos()

In [ ]:
df = pd.merge_asof(qos, bw, tolerance = 300).dropna()
df = df.query("ul < 100").groupby("ul").mean()

In [ ]:
fig, axes = plt.subplots(4, 1, figsize = (6, 8), sharex = True)
label = {"bitrate" : "Bitrate [Mpbs]", "loss" : "Loss [%]", 
         "fps" : "FPS", "resx" : "Horizontal\nResolution"}


for y, ax in zip(["bitrate", "loss", "fps", "resx"], axes):

    df[y].plot(ax = ax, color = chred)
    ax.set_ylabel(label[y], labelpad = 10)
    ax.set_xlabel("Upload Bandwidth [Mbps]")
    
fig.savefig("zoom_perf.pdf")